## Bibliotecas

In [1]:
import scipy.io
import pickle
import numpy as np
import pandas as pd
from joblib import load

## Funções úteis

In [2]:
def count_values(data):
  unique, counts = np.unique(data, return_counts=True)

  data_dict = dict(zip(unique, counts))
  return data_dict

In [3]:
E1 = scipy.io.loadmat(f"./data/S1/S1_E1_A1.mat") # Exercise A
E2 = scipy.io.loadmat(f"./data/S1/S1_E2_A1.mat") # Exercise B
E3 = scipy.io.loadmat(f"./data/S1/S1_E3_A1.mat") # Exercise C

print(count_values(E1["stimulus"]))
print(count_values(E2["stimulus"]))
print(count_values(E3["stimulus"]))
E1["stimulus"]

{0: 49599, 1: 6753, 2: 6656, 3: 6613, 4: 6654, 5: 6782, 6: 6776, 7: 6776, 8: 6795, 9: 6626, 10: 6696, 11: 6773, 12: 6768}
{0: 65453, 1: 6822, 2: 6787, 3: 6668, 4: 6788, 5: 6666, 6: 6808, 7: 6795, 8: 6808, 9: 6796, 10: 6609, 11: 6618, 12: 6777, 13: 6784, 14: 6672, 15: 6662, 16: 6623, 17: 6765}
{0: 104387, 1: 6624, 2: 6777, 3: 6620, 4: 6787, 5: 6611, 6: 6628, 7: 6661, 8: 6819, 9: 6606, 10: 6792, 11: 6791, 12: 6799, 13: 6789, 14: 6652, 15: 6605, 16: 6610, 17: 6789, 18: 6795, 19: 6776, 20: 6597, 21: 6666, 22: 6590, 23: 6601}


array([[ 0],
       [ 0],
       [ 0],
       ...,
       [12],
       [12],
       [12]], dtype=int8)

## Conversão dos dados para python

In [5]:
n_subjects = 10
files = [f"S{x}" for x in range(1,n_subjects+1)] 
data = dict()
for file in files:
  E1 = scipy.io.loadmat(f"./data/{file}/{file}_E1_A1.mat") # Exercise A
  E2 = scipy.io.loadmat(f"./data/{file}/{file}_E2_A1.mat") # Exercise B
  E3 = scipy.io.loadmat(f"./data/{file}/{file}_E3_A1.mat") # Exercise C
  
  # Movimento 0 é o movimento de repouso, então é subtraído 12 e 29 para os experimentos 2 e 3
  E2["stimulus"][np.where(E2["stimulus"]==0)] = E2["stimulus"][np.where(E2["stimulus"]==0)]-12
  E3["stimulus"][np.where(E3["stimulus"]==0)] = E3["stimulus"][np.where(E3["stimulus"]==0)]-29
  
  data[file] = {
    'emg':np.concatenate([E1["emg"], E2['emg'], E3['emg']]),
    'label':np.concatenate([E1["stimulus"], E2["stimulus"]+12, E3["stimulus"] +29]),
    'info':{
      'age':E1['age'],
      'gender':E1['gender'], 
      'n_subjects':10,
      'weight':E1['weight'], 
      'height':E1['height'],
      'fs':E1["frequency"][0][0]
    }, 
    'description': "Contiene datos de EMG de 10 sujetos, 3 experimentos por cada sujeto (E1, E2 y E3) (A, B y C de la figura), cada experimento contiene diferentes ejercicios (A:12, B:17, C:23)."
  }

In [6]:
with open("./data/MyoArmband_data.pickle", "wb") as f:
  pickle.dump(data, f)

## Mapeamento dos movimentos
Para mais detalhes sobre os movimentos, acesse o [NinaPro](https://ninapro.hevs.ch/instructions/DB5.html)

In [9]:
movements_dict = {
  0: "Rest",
  1: "Index flexion",
  2: "Index extension",
  3: "Middle flexion",
  4: "Middle extension",
  5: "Ring flexion",
  6: "Ring extension",
  7: "Little_flexion",
  8: "Little extension",
  9: "Thumb adduction",
  10: "Thumb abduction",
  11: "Thumb flexion",
  12: "Thumb extension",
  13: "Thumb up",
  14: "Extesion index and middle, flexion the others",
  15: "Flexion ring and little, extension the others",
  16: "Thumb opposing base little of finger",
  17: "Abduction of all fingers",
  18: "Fingers flexed together in fist",
  19: "Pointing index",
  20: "Adductiion of extended fingers",
  21: "Wrist supination (axis: middle finger)",
  22: "Wrist pronation (axis: middle finger)",
  23: "Wrist supination (axis: little finger)",
  24: "Wrist pronation (axis: little finger)",
  25: "Wrist flexion",
  26: "Wrist extension",
  27: "Wrist radial deviation",
  28: "Wrist ulnar deviation",
  29: "Wrist extension with closed hand",
  30: "Large diameter grasp",
  31: "Small diameter grasp (power grip)",
  32: "Fixed hook grasp",
  33: "Index finger extension grasp",
  34: "Medium wrap",
  35: "Ring grasp",
  36: "Prismatics four fingers grasp",
  37: "Stick grasp",
  38: "Writting tripod grasp",
  39: "Power sphere grasp",
  40: "Three finger sphere grasp",
  41: "Precision sphere grasp",
  42: "Trripod grasp",
  43: "Prismatic pinch grasp",
  44: "Tip pinch grasp",
  45: "Quadpod grasp",
  46: "Lateral grasp",
  47: "Parallel extension grasp",
  48: "Extension type grasp",
  49: "Power disk grasp",
  50: "Open a bottle with a tripod grasp",
  51: "Turn a screw",
  52: "Cut something",
}

## Separar os dados por movimento
obs: rodar o notebook featuresCalc.ipynb antes para gerar os dados com as features

In [10]:
df = load("../data/features_df.joblib")
df = df.round(2)
df

FileNotFoundError: [Errno 2] No such file or directory: '../data/features_df.joblib'

In [18]:
path = "../../prototype/data/features"
for i in movements_dict:
  df_i = df[df['mov'] == i]
  df_i.to_csv(f"{path}/{i}.csv", index=False)